In [ ]:
import os
os.environ['PYTHONHASHSEED']=str(1)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import cv2
import keras
import random
from sklearn.metrics import classification_report
import time
from keras.models import Sequential
from keras import regularizers
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, MaxPooling2D

In [ ]:
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   tf.random.set_seed(1)
   np.random.seed(1)
   random.seed(1)
reset_random_seeds()

In [ ]:
ruta_drive = "C:/Users/Emilio/TESINA"

In [ ]:
directorio_dataset = "C:/Users/Emilio/TESINA/DATASET_RASPBERRY"

dir_entrenamiento = os.path.join(directorio_dataset, 'entrenamiento')
dir_validacion = os.path.join(directorio_dataset, 'validacion')
dir_prueba = os.path.join(directorio_dataset, 'prueba')

TAMANIO_BATCH = 32
TAMANIO_IMG = (165,165)
epocas = 8
dataset_entrenamiento = tf.keras.utils.image_dataset_from_directory(dir_entrenamiento,
                                                                    labels='inferred',
                                                                    label_mode='binary',
                                                                    shuffle=True,
                                                                    batch_size=TAMANIO_BATCH,
                                                                    image_size=TAMANIO_IMG)

In [ ]:
dataset_validacion = tf.keras.utils.image_dataset_from_directory(dir_validacion,
                                                                 labels='inferred',
                                                                 label_mode='binary',
                                                                 shuffle=True,
                                                                 batch_size=TAMANIO_BATCH,
                                                                 image_size=TAMANIO_IMG)

In [ ]:
dataset_prueba = tf.keras.utils.image_dataset_from_directory(dir_prueba,
                                                             labels='inferred',
                                                             label_mode='binary',
                                                             shuffle=True,
                                                             batch_size=TAMANIO_BATCH,
                                                             image_size=TAMANIO_IMG,
                                                            )

In [ ]:
class_names = dataset_entrenamiento.class_names
plt.figure(figsize=(10, 10))
for images, labels in dataset_entrenamiento.take(1):
  print(labels.shape)
  labels
  for i in range(8):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i].numpy()[0].astype('uint8')])
    plt.axis("off")

In [ ]:
print('Lotes de validación: %d' % tf.data.experimental.cardinality(dataset_validacion))
print('Lotes de prueba: %d' % tf.data.experimental.cardinality(dataset_prueba))

In [ ]:
#Config del dataset para mejorar el rendimiento
AUTOTUNE = tf.data.AUTOTUNE

dataset_entrenamiento = dataset_entrenamiento.prefetch(buffer_size=AUTOTUNE)
dataset_validacion = dataset_validacion.prefetch(buffer_size=AUTOTUNE)
dataset_prueba = dataset_prueba.prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Aumentación del conjunto de entrenamiento, aplicando distintos tipos de transformaciones para generar nuevas muestras del mismo.
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [ ]:
for image, _ in dataset_entrenamiento.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

**Modelo de 4 capas - Relu**

In [ ]:
def create_model():
    model = Sequential()
    
    model.add(Conv2D(16, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (165,165,3)))
    model.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
    model.add(MaxPool2D(pool_size = [3,3]))
        
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), 
                  loss = tf.keras.losses.BinaryCrossentropy(), 
                  metrics = ["accuracy", keras.metrics.Recall(), keras.metrics.Precision()])
    
    print("MODELO CREADO")
    model.summary()
    
    return model

def fit_model():
    model_hist = model.fit(dataset_entrenamiento, epochs=epocas, validation_data=dataset_validacion)
    return model_hist 

In [ ]:
model = create_model()
tiempoInicio = time.time()
curr_model_hist = fit_model()

tiempoTranscurrido = (time.time() - tiempoInicio)
tiempoTranscurrido = round(tiempoTranscurrido,1)
print ('\nTiempo de ejecución:', int(tiempoTranscurrido/60), 'minutos')

metricas = model.evaluate(dataset_prueba)

In [ ]:
print("Eficacia del test = ", "{:.2f}%".format(metricas[1]*100),
      "\nPérdida del test = " ,"{:.2f}".format(metricas[0]),
      "\nRecall del test = " ,"{:.2f}%".format(metricas[2]*100),
      "\nPrecisión del test = " ,"{:.2f}%".format(metricas[3]*100))

In [ ]:
acc = curr_model_hist.history['accuracy']
val_acc = curr_model_hist.history['val_accuracy']

loss = curr_model_hist.history['loss']
val_loss = curr_model_hist.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Exactitud del entrenamiento')
plt.plot(val_acc, label='Exactitud de la validación')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Exactitud del entrenamiento y validación')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Pérdida del entrenamiento')
plt.plot(val_loss, label='Pérdida de la valicación')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Pérdida del entrenamiento y validación')
plt.xlabel('Época')
plt.show()

In [ ]:
# Obtiene un lote del conjunto de prueba para su predicción
image_batch, label_batch = dataset_prueba.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()
predictions = np.round(predictions,1).astype(np.int32)

print('Predicciones:\n', predictions)
print('Etiquetas:\n', label_batch.flatten().astype(np.int32))
print(classification_report(label_batch, predictions))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

matriz = tf.math.confusion_matrix(
              label_batch,
              predictions,
              num_classes=None,
              weights=None,
              dtype=tf.dtypes.int32,
              name=None
          )
cf_matrix = confusion_matrix(label_batch, predictions)
sns.heatmap(cf_matrix, annot=True)